# Import libraries

In [83]:
import pandas as pd
pd.options.display.max_columns = 200

import requests as req
import json
import boto3
import io

import sys
import logging
import os
logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

import itertools

# Constants

In [84]:
RW_API_TOKEN = ""

# Grab website metadata

In [85]:
# Base URL for getting dataset metadata from RW API
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&published=true&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000, "language":"en"}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[dset["id"]] = {
        "name":atts["name"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

logging.info("Number of datasets: " + str(current_datasets_on_api.shape[0]))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.resourcewatch.org
DEBUG:urllib3.connectionpool:https://api.resourcewatch.org:443 "GET /v1/dataset?sort=slug,-provider,userId&status=saved&published=true&includes=metadata,vocabulary,widget,layer&application=rw&page%5Bsize%5D=1000&language=en HTTP/1.1" 200 502567
INFO:root:Number of datasets: 225


In [ ]:
current_datasets_on_api.iloc[0]

# Pull metadata, layer, widget info

In [13]:
def pick_layer_info(layer, f):
    s_atts = layer['attributes']
    t_atts = ['name', 'description']
    f.write('*layer id*: {}\n'.format(layer['id']))
    for att in t_atts:
        if att in s_atts:
            f.write('*{}*: {}\n'.format(att, s_atts[att]))
    f.write('~end_layer~\n\n')

def pick_widget_info(widget, f):
    s_atts = widget['attributes']
    t_atts = ['name', 'description']
    f.write('*widget id*: {}\n'.format(widget['id']))
    for att in t_atts:
        if att in s_atts:
            f.write('*{}*: {}\n'.format(att, s_atts[att]))
    f.write('~end_widget~\n\n')
    
def pick_metadata_info(metadata, f):
    if not len(metadata):
        return

    s_atts = metadata['attributes']
    t_top_atts = ['description']
    t_info_atts = ['cautions', 'functions', 'citation']
    f.write('*metadata id*: {}\n'.format(metadata['id']))
    
    for att in t_top_atts:
        if att in s_atts:
            f.write('*{}*: {}\n'.format(att, s_atts[att]))
    
    for att in t_info_atts:
        if att in s_atts['info']:
            f.write('*{}*: {}\n'.format(att, s_atts['info'][att]))
    f.write('~end_metadata~\n\n')
    
def record_dataset_info(info, f):
    '''Includes information on metadata, layers, widgets, and tags'''
    ds = info[0]
    info = info[1]
    
    name = info['name']
    metadata = info['metadata']
    layers = info['layers']
    widgets = info['widgets']
    tags = info['tags']
    
    f.write('** NEW DATASET **\n')
    f.write('ds name: {}\n'.format(name))
    f.write('ds id: {}\n'.format(ds))
    f.write('ds on site: https://staging.resourcewatch.org/data/explore/{}\n\n'.format(ds))
    
    f.write('~ metadata ~\n\n')
    list(map(lambda m: pick_metadata_info(m, f), metadata))
    
    f.write('~ layers ~\n\n')
    list(map(lambda l: pick_layer_info(l,f),layers))
    
    f.write('~ widgets ~\n\n')
    list(map(lambda w: pick_widget_info(w,f),widgets))


In [14]:
try:
    os.unlink('all_info.txt')
except:
    pass

f = open('all_info.txt', 'a')

list(map(lambda i: record_dataset_info(i, f), current_datasets_on_api.iterrows()))
    
f.close()

# Prove inverse works

In [106]:
def isplit(iterable,splitters):
    return [list(g) for k,g in itertools.groupby(iterable,lambda x:x in splitters) if not k]

def patch_layers(info, ds, send=True):
    
    rw_api_url = 'https://api.resourcewatch.org/v1/dataset/{}/layer'.format(ds)
    try:
        old_layers = req.get(rw_api_url).json()['data']
    except:
        logging.info('No layers to work on')
        return (ds, 'no layers to update')
        
    edit_layers = isplit(info, '~end_layer~')
    edit_layer_info = {}
    for edit_layer in edit_layers:
        e_id = edit_layer[0].split(':')[1].strip()
        edit_layer_info[e_id] = {
            'name':':'.join(edit_layer[1].split(':')[1:]).strip(),
            'description':':'.join(edit_layer[2].split(':')[1:]).strip()
        }
    
        
    for old_layer in old_layers:
        layer_id = old_layer['id']
        if layer_id in edit_layer_info:
            logging.info('Layer ID: {}'.format(layer_id))

            new_layer = old_layer.copy()
            new_atts = old_layer['attributes'].copy()
            edit_atts = edit_layer_info[layer_id]
            
            new_atts.update(name = edit_atts['name'], description= edit_atts['description'])
            new_layer.update(attributes = new_atts)
            #logging.info('New layer: {}'.format(new_layer))
            
            headers = {
                'content-type': "application/json",
                'authorization': "Bearer {}".format(RW_API_TOKEN)
            }
            patch_url = 'https://api.resourcewatch.org/v1/dataset/{}/layer/{}'.format(ds, layer_id)

            logging.debug('URL: {}'.format(patch_url))
            # Tried json.dumps(layer) and that didn't work
            if send: 
                res = req.request("PATCH", patch_url, data=json.dumps(new_atts), headers = headers)
                logging.info('Layer load ok? {}'.format(res.ok))
                if not res.ok:
                    return (ds, layer_id, res.text)
                #logging.info('Layer load Response: {}'.format(res.text))
    

    
def patch_metadata(info, ds, send=True):
    rw_api_url = 'https://api.resourcewatch.org/v1/dataset/{}/metadata'.format(ds)
    
    payload = { "application":"rw", "language":"en"}
    old_metadata = req.get(rw_api_url,params=payload).json()['data']
    logging.info('Num old metadatas: {}'.format(len(old_metadata)))
    
    edit_metadata = isplit(info, '~end_metadata~')
    if len(edit_metadata) == 0:
        logging.info('No metadata to update')
        return (ds, 'no metadata')
    else:
        edit_metadata = edit_metadata[0]
        if len(edit_metadata) < 4:
            logging.info('All fields not present')
            return (ds, 'not enough fields')
    
    if ((len(edit_metadata[1].split(':'))>2) or (len(edit_metadata[2].split(':'))>2) or (len(edit_metadata[3].split(':'))>2) or (len(edit_metadata[4].split(':'))>2)):

        edit_metadata_info = {}
        e_id = edit_metadata[0].split(':')[1].strip()
        edit_metadata_info[e_id] = {
            'description':':'.join(edit_metadata[1].split(':')[1:]).strip(),
            'cautions':':'.join(edit_metadata[2].split(':')[1:]).strip(),
            'functions':':'.join(edit_metadata[3].split(':')[1:]).strip(),
            'citation':':'.join(edit_metadata[4].split(':')[1:]).strip()
        }
        if len(edit_metadata[1].split(':')) > 2:
            logging.info('Metadata info: {}'.format(str(edit_metadata_info[e_id])))   

        if len(old_metadata) == 0:
            create_new_metadata = True
            old_metadata = [{'id':key, 'attributes':{'info':values}} for key, values in edit_metadata_info.items()][0]
        else:
            create_new_metadata = False
            old_metadata = old_metadata[0]

        metadata_id = old_metadata['id']
        if (metadata_id in edit_metadata_info) or create_new_metadata:
            logging.info('Metadata ID: {}'.format(metadata_id))

            new_metadata = old_metadata.copy()
            new_atts = old_metadata['attributes'].copy()
            new_info = new_atts['info'].copy()
            edit_atts = edit_metadata_info[metadata_id]

            new_atts.update(description= edit_atts['description'])
            new_info.update(cautions = edit_atts['cautions'], functions = edit_atts['functions'])
            new_atts.update(info = new_info)

            new_metadata.update(attributes = new_atts)
            #logging.info('New metadata: {}'.format(new_metadata))

            headers = {
                'content-type': "application/json",
                'authorization': "Bearer {}".format(RW_API_TOKEN)
            }

            url = 'https://api.resourcewatch.org/v1/dataset/{}/metadata'.format(ds)

            logging.debug('URL: {}'.format(url))

            if send:
                res = req.request("POST", url, data=json.dumps(new_atts), headers = headers)
                if res.ok:
                    logging.info('New metadata uploaded')
                else:
                    logging.info('Whoops, already exists! Updating metadata.')
                    res = req.request("PATCH", url, data=json.dumps(new_atts), headers = headers)
                    logging.info('Ok now?: {}'.format(res.ok))

                #logging.info('Metadata load Response: {}'.format(res.text))

def structure_patchs(lst):
    if not len(lst):
        return None
    
    #logging.info(lst)
    ds = lst[1].split(':')[1].strip()
    #logging.info(ds)
    
    metadata_start = lst.index('~ metadata ~')
    layers_start = lst.index('~ layers ~')
    widgets_start = lst.index('~ widgets ~')
    
    metadata = lst[metadata_start:layers_start]
    layers = lst[layers_start:widgets_start]
    widgets = lst[widgets_start:]
    
    #skipped_meta = patch_metadata(metadata[1:], ds)
    skipped_layers = patch_layers(layers[1:], ds)
    #patch_widgets(widgets[1:], ds)
    
    #return (skipped_meta, skipped_layers)
    return skipped_layers

def send_patches(txt):
    by_ds = txt.split('** NEW DATA SET **')
    #logging.info('By DS: {}'.format(by_ds))
    by_line = list(map(lambda i: i.split('\n'), by_ds))
    #logging.info('By Line: {}'.format(by_line))
    clean_lines = list(map(lambda l: list(filter(lambda i: len(i)>0,l)), by_line))
    #logging.info('Clean LinesS: {}'.format(clean_lines))
    skipped_patches = list(map(structure_patchs, clean_lines))
    return skipped_patches

In [107]:
with open('edited_metadata_and_layers.txt', 'r') as src:
    skipped_patches = send_patches(src.read())

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.resourcewatch.org
DEBUG:urllib3.connectionpool:https://api.resourcewatch.org:443 "GET /v1/dataset/a290675c-9528-4a51-8201-f6c2d7848744/layer HTTP/1.1" 200 1161
INFO:root:Layer ID: 09aa80be-d7c8-4008-98fb-8adbd3a2e05b
DEBUG:root:URL: https://api.resourcewatch.org/v1/dataset/a290675c-9528-4a51-8201-f6c2d7848744/layer/09aa80be-d7c8-4008-98fb-8adbd3a2e05b
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.resourcewatch.org
DEBUG:urllib3.connectionpool:https://api.resourcewatch.org:443 "PATCH /v1/dataset/a290675c-9528-4a51-8201-f6c2d7848744/layer/09aa80be-d7c8-4008-98fb-8adbd3a2e05b HTTP/1.1" 200 2663
INFO:root:Layer load ok? True
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.resourcewatch.org
DEBUG:urllib3.connectionpool:https://api.resourcewatch.org:443 "GET /v1/dataset/a00c3514-94ba-48bd-bf56-a2e65722be34/layer HTTP/1.1" 200 794
INFO:root:Layer ID: 7cbd8ad3-cb16-4a6c-bca8-c5378528e0

In [108]:
list(filter(lambda item: item, skipped_patches))

[('c887d4c9-7747-4d35-9715-310fd219143b', 'no layers to update'),
 ('60be01b0-99fb-459c-8a08-b934270f8c4b',
  'd0ecc6fb-cc31-461c-8aa6-7a1a2b906c8c',
  '\n<html><head>\n<meta http-equiv="content-type" content="text/html;charset=utf-8">\n<title>502 Server Error</title>\n</head>\n<body text=#000000 bgcolor=#ffffff>\n<h1>Error: Server Error</h1>\n<h2>The server encountered a temporary error and could not complete your request.<p>Please try again in 30 seconds.</h2>\n<h2></h2>\n</body></html>\n'),
 ('3c2fad43-debb-4351-b9d6-b81b71317f8d', 'no layers to update'),
 ('55914c68-508c-4125-9c11-f95ca89285fb', 'no layers to update'),
 ('b0f859ce-f13b-462e-9063-ebc68ed88420', 'no layers to update'),
 ('f4875c0e-9b2b-4f86-a338-f63dc5b33863', 'no layers to update'),
 ('b8307c16-fd77-4e35-9b68-8726a025f401',
  'f66acc52-455b-44d9-ad48-d10a301a0f4f',
  '\n<html><head>\n<meta http-equiv="content-type" content="text/html;charset=utf-8">\n<title>502 Server Error</title>\n</head>\n<body text=#000000 bgcol

In [109]:
with open('record_of_skipped_layers.txt', 'w') as f:
    f.write(str(list(filter(lambda item: item, skipped_patches))))

In [79]:
links = ['https://api.resourcewatch.org/v1/dataset/{}'.format(ds[0][0]) for ds in skipped_patches if ds[0]]
list(map(print, links))

TypeError: 'NoneType' object is not subscriptable

# Experimentation below

In [ ]:
load_info_sample = '** NEW DATASET **\nds name: soc.069 Political Violence Risk Assessment\nds id: 25eebe25-aaf2-48fc-ab7b-186d7498f393\nds on site: https://staging.resourcewatch.org/data/explore/25eebe25-aaf2-48fc-ab7b-186d7498f393\n\n~ metadata ~\n\nmetadata id: 59debf249d46b60010c48aad\ndescription: CHANGE ME The Political Violence Risk Assessment data set was originally published by the Hague Centre for Strategic Studies. Its purpose is to forcast the onset of large-scale political violence. Countries are evaluated to determine their "risk" of experiencing a large-scale political violence event in the next month using structural data (infant mortality, regime type, conflicts in neighboring countries, and state-led discrimination) and automated event data (protests, assaults, and expressions of disapproval) from the previous month, which are are combined in a logistic regression model. The score is normalized between 0 and 100 and indicates the percentile rank of probability of onset. Infant mortality data are from World Development Indicators; other structural data are from the Center for Systemic Peace; event data are from the Global Database of Events, Language, and Tone; and normalization and aggregation were done by the HCSS. Resource Watch shows only a subset of the dataset. For access to the full dataset and additional information, see the Learn More link.\ncautions: The data page provides no formal cautions.\nfunctions: Countries most at risk for political violence\n~end_metadata~\n\nmetadata id: 59e125369817d60012ba095f\ndescription: Both structural data (infant mortality, regime type, conflicts in neighboring countries, and state-led discrimination) is combined with automated event data (protests, assaults, and expressions of disapproval) of the previous month are combined in a logistic regression model to assess the risk of civil war onset in the next month. The score is normalized between 0 and 100 and indicates the percentile rank of probability of onset. Infant mortality data is from WDI, other structural data is from Center for Systemic Peace, event data is from GDELT, normalization and aggregation were done by HCSS.\ncautions: The data page provides no formal cautions. \nfunctions: Countries most at risk for political violence\n~end_metadata~\n\n~ layers ~\n\nlayer id: 1f8090c0-6594-45e2-b1df-1cbed74f12a8\nname: Political Violence Risk Assessment\ndescription: Risk score on a country level indicating the percentile rank of the likelihood of civil war onset during the next 30 days, updated daily. Both structural indicators and dynamic, event-based variables are used in regression.  The higher the value, the more at risk of civil war.\n~end_layer~\n\n~ widgets ~\n\nwidget id: 54160dca-ecb3-4d62-b80e-95970e8e834d\nname: Political Violence Risk Assessment\ndescription: \n~end_widget~\n\n'
send_patchs(load_info_sample)

In [ ]:
list(map(len, current_datasets_on_api['metadata']))

# Pull out all layer names and definitions
* Link to data set detail page
* For each layer:
* * Layer name
* * Layer description


In [ ]:
try:
    os.unlink('layer_info.txt')
except:
    pass


### NOTE: use option 'a' to append to a file. 'w' would overwrite
f = open('layer_info.txt', 'a')

def pick_layer_info(layer, f):
    s_atts = layer['attributes']
    t_atts = ['name', 'description']
    f.write('layer id: {}\n'.format(layer['id']))
    for att in t_atts:
        if att in s_atts:
            f.write('{}: {}\n'.format(att, s_atts[att]))
    f.write('\n')
            
            
def record_layers(info, f):
    ds = info[0]
    layers = info[1]
    f.write('** NEW DATASET **\n')
    f.write('ds id: {}\n'.format(ds))
    f.write('ds on site: https://staging.resourcewatch.org/data/explore/{}\n\n'.format(ds))
    f.write('~ layers ~\n\n')
    list(map(lambda l: pick_layer_info(l,f), layers))
    f.write('\n')

list(map(lambda i: record_layers(i,f), current_datasets_on_api['layers'].items()))

f.close()

# Pull out all widget names and definitions
* Link to data set detail page
* For each widget:
* * Widget name
* * Widget description

In [ ]:
try:
    os.unlink('widget_info.txt')
except:
    pass

f = open('widget_info.txt', 'a')

def pick_widget_info(widget, f):
    s_atts = widget['attributes']
    t_atts = ['name', 'description']
    f.write('widget id: {}\n'.format(widget['id']))
    for att in t_atts:
        if att in s_atts:
            f.write('{}: {}\n'.format(att, s_atts[att]))
    f.write('\n')
            
            
def record_widgets(ds, info, f):
    ds = info[0]
    widgets = info[1]
    f.write('** NEW DATASET **\n')
    f.write('ds id: {}\n'.format(ds))
    f.write('ds on site: https://staging.resourcewatch.org/data/explore/{}\n\n'.format(ds))
    f.write('~ widgets ~\n\n')
    list(map(lambda l: pick_widget_info(l,f), widgets))
    f.write('\n')

list(map(lambda i: record_widgets(i,f), current_datasets_on_api['widgets'].items()))

f.close()

In [ ]:
def patch_widgets(info, ds):
    
    rw_api_url = 'https://api.resourcewatch.org/v1/dataset/{}/widget'.format(ds)
    old_widgets = req.get(rw_api_url).json()['data']
    
    edit_widgets = isplit(info, '~end_widget~')
    edit_widget_info = {}
    for edit_widget in edit_widgets:
        e_id = edit_widget[0].split(':')[1].strip()
        edit_widget_info[e_id] = {
            'name':edit_widget[1].split(':')[1].strip(),
            'description':edit_widget[2].split(':')[1].strip()
        }
        
    for old_widget in old_widgets:
        widget_id = old_widget['id']
        if widget_id in edit_widget_info:
            logging.info('Widget ID: {}'.format(widget_id))

            new_widget = old_widget.copy()
            new_atts = old_widget['attributes'].copy()
            edit_atts = edit_widget_info[widget_id]
            
            new_atts.update(name = edit_atts['name'], description= edit_atts['description'])
            new_widget.update(attributes = new_atts)
            logging.info('New widget: {}'.format(new_widget))
            
            patch_url = 'https://api.resourcewatch.org/v1/dataset/{}/widget/{}'.format(ds, widget_id)
            headers = {
                'content-type': "application/json",
                'authorization': "Bearer {}".format(RW_API_TOKEN)
            }
            logging.debug('URL: {}'.format(patch_url))
            # Tried json.dumps(layer) and that didn't work
            res = req.request("PATCH", patch_url, data=json.dumps(new_atts), headers = headers)
            logging.info('Widget load Response: {}'.format(res.text))